Cells below are used in the matching of records in source_efsa_raw to clowder information in the index files.

Major goals are to...

    Initialize the kernel by importing modules, loading the environment file, and the necessary excel sheets as dataframes
    
    Create a "key" that holds only unique entries in source_efsa_raw
    
    Match entries in the key to the index files and fill in clowder information (id and file name) based on the record_url and long_ref fields
    
    Matching key entries to source_efsa_raw entries and copy over the clowder information
    
    Output a spreadsheets of key checkpoints for inspection


In [ ]:
#Script to match index entries to EFSA data

import pandas as pd #For dealing with excel spreadsheets as dataframes
import numpy as np #Basic math and functions. Could be useful
import requests #For API interactions
import time as t #sleep times
from dotenv import load_dotenv #environment files for credentials. Make sure to pip install dotenv first
import os #For accessing environment file

#environment file with credentials
load_dotenv()

In [ ]:
#Load in the reference and raw excel sheets as pandas dataframes.

raw = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\source_efsa_raw.xlsx")
master_ref_info = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\index files\CCTE_Deliverable_RefinfoSheet_Master.xlsx")
master_track = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\index files\CCTE_PDF_MasterTrackingSheet_Deliverable.xlsx", sheet_name = 1)
ref_info = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\index files\CCTE_TSCAPOCDeliverable_RefinfoSheet.xlsx")
toxval = pd.read_excel(r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\index files\ToxValDB TSCA POC references for ICF_MasterTracking_Deliverable.xlsx", sheet_name = 1)

In [ ]:
#Manipulate the column headers a bit. This is to ensure the headers have the same name and convention (url vs URL and Source Hash vs source_hash)
master_ref_info.columns = master_ref_info.columns.str.lower().str.replace(' ','_')
master_track.columns = master_track.columns.str.lower().str.replace(' ','_')
ref_info.columns = ref_info.columns.str.lower().str.replace(' ','_')
toxval.columns = toxval.columns.str.lower().str.replace(' ','_')

#Dictionary to hold all of the dataframes
index_files = {'master_ref_info':master_ref_info, 'master_track':master_track,'ref_info':ref_info, 'toxval':toxval}

In [ ]:
#Efsa raw is REALLY big with lots of duplicates. Lets make a "key" dataframe that holds only the unique entries in efsa raw

#array to hold the unique urls in efsa raw
unique_url = []

#emtpy dictionary to initialize the key dataframe
d = {}
#inital empty dataframe
key = pd.DataFrame(data = d)
j = 0

#look through all the urls in raw. save them to the unique_url array (used for checking later) and put them in the key 
for i in range(len(raw)):
    url = raw.loc[i,'record_url']
    #long_ref = raw.loc[i,'long_ref']
    #year = str(raw.loc[i,'year'])
    if url not in unique_url:
        unique_url.append(url)
#copy the data in efsa raw for each unique url into identical columns in the key dataframe
        for column in raw.columns:
            key.loc[j,column] = raw.loc[i,column]
        j += 1

#sanity checks
#print(len(unique_url))
#print(len(key))
#key.loc[[0]]

In [ ]:
#Search through the index files for each unique entry in the key starting with the URLs.
#Some manual checking, inspection, and comparison was done as the key was being made.
#We will drop some of these columns in the final version of the key.


#Keep track of the index file we found each entry in
blank = ['' for x in range(len(key))]
key['found_in'] = blank

for i in range(len(key)):
#Progress check to make sure we are moving along
    if i%100 == 0:
        print(i)
#First, search for direct url matches.
    url = key.loc[i,'record_url']
    raw_year = int(key.loc[i,'year'])
    for name in index_files:
#Look through each index file for a match in the 'url' field
        source = index_files[name]
        source = source.fillna('')
        other = source[source['url'].str.match(url)]
#If there is a match, we have found the index file to reference! We can stop the loop.
        if other.empty == False:
            key.loc[i,'found'] = 'found'
            break
#If you cannot find a match in any of the index files, move on to the next url and fill that in the key
    if other.empty == True:
        #missing1.append(url)
        key.loc[i,'found'] = 'not found'
        key.loc[i,'index_long_ref'] = 'not found'
        continue
#I noticed some urls have several matches in an index file. The index files include duplicates.
    idx = 0
#If there are several matches, look at each one and see if the index file lists it as a duplicate.
    if len(other) > 1:
        for idx in range(len(other.index)):
            status = other.loc[other.index[idx],'duplicate']
#We will also keep track of the year listed in the index file. Some of these are inconsistently populated.
            index_year = int(other.loc[other.index[idx],'year'])
#Once we have found the primary reference, we can break out and work with it
            if status.lower() == 'primary':
                break
#If the index file only has duplicates, it has the hash for the primary. Look there.
    try:
        status = other.loc[other.index[idx],'duplicate']
        if status.lower() == 'duplicate':
#Extract the record hashes listed in the excel sheets. If the match is as duplicate: Look in the primary section
            record_hash = other.loc[other.index[idx],'primary_reference_for_duplicate_set']
        else:
#If the match is a primary, look at its record hash
            record_hash = other.loc[other.index[idx],'record_source_hash']
    except:
#Not all index files have the duplicate field. I can only assume they only have primaries.
        record_hash = other.loc[other.index[idx],'record_source_hash']
#Start filling in comparison information like the long refs, full citations, etc. from both raw and the index file
    key.loc[i,'index_long_ref'] = other.loc[other.index[idx],'long_ref']
    key.loc[i,'index_citation'] = other.loc[other.index[idx],'citation']
    key.loc[i,'found_in'] = key.loc[i,'found_in'] + ' ' + name
    index_year = int(other.loc[other.index[idx],'year'])
    key.loc[i,'index_year'] = index_year          
    key.loc[i,'record_hash'] = record_hash
#Search for the record hash using the clowder api and examine the json results
    url_start = r'https://clowder.edap-cluster.com/api/search?query='
    url_end = r'&order=asc'
    api_key = os.environ.get('apiKey')
    t.sleep(0.25) #Courtesy sleep time because we're respectful in this house
    response = requests.get(url_start+record_hash+url_end, headers = {'X-API-Key': api_key})
    json = response.json()
#Walk through the json to find the clowder id and file name. Then populate their respective fields in the key
    try:
        clowder_id = json['results'][0]['id']
        file_name = json['results'][0]['name']
        key.loc[i,'file_name'] = file_name
        key.loc[i,'clowder_id'] = clowder_id
        key.loc[i,'found'] = 'found'
#Make note of any exceptions (clowder searches with no results).These record hashes are not in clowder.
    except:
        key.loc[i,'clowder_id'] = ''
        key.loc[i,'file_name'] = 'record match but document not found in clowder'
        key.loc[i,'found'] = 'found in index files but not found in clowder'
    if raw_year != index_year:
        key.loc[i,'found'] = 'found but index year and raw year do not match' 
#Save the key as an excel file to view and make sure everything looks alright.
out_path = r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\source_efsa_key_mmille16_09212022.xlsx"
with pd.ExcelWriter(out_path) as writer:
    key.to_excel(writer, index = False)
print('All done!!')

#Count all of the unique clowder ids and how many unique documents were not found

#print(list(key['found']).count('not found'))
#print(len(np.unique(list((key['clowder_id'])))))

In [ ]:
#URL matching probably caught most matches, but lets look for long ref matches just in case any were missed

#This array will hold new matches so that I can manually inspect once finished
new_matches = []


for i in range(len(key)):
#Progress check while we look through the key
    if i%100 == 0:
        print(i)
#Only do long ref matching if the document was marked as 'not found' from the url matching
    if key.loc[i,'found'] != 'not found':
        continue
#Then extract the long ref field
    long_ref = str(key.loc[i,'long_ref'])
    
#Now we reuse the search piece of code from before. Only now, we look at the long ref field instead of the url field
    for name in index_files:
        source = index_files[name]
        source = source.fillna('')
#Using str.contains as opposed to str.match becuase some of these long_refs have a variety of regex symbols
#Can't turn off regex for str.match but can for str.contains
        other = source[source['long_ref'].str.contains(long_ref, case = False, regex = False)]
#Stop looking once we have found a match
        if other.empty == False:
            break
#If we looked through all the index files and found no match, move on
    if other.empty == True:
        continue
#More rigorous testing. Since we used 'contains', we might not have exact matches.
    if long_ref not in new_matches:
        index_long_ref = other.loc[other.index[0],'long_ref']
        index_citation = other.loc[other.index[0],'citation']
        raw_year = str(key.loc[i,'year'])
        raw_year = raw_year.replace('.0','')
#Now that we know the index long ref CONTAINS the key long ref, see if they are the exact same and made in the same year
        if long_ref.lower() == index_long_ref.lower() and raw_year in index_citation:
#If all those checks passed, we found a new match and we can repeat the population of the key
            new_matches.append(long_ref)
#Pull the record hash and search for it in the clowder api
            record_hash = other.loc[other.index[0],'record_source_hash']
            url_start = r'https://clowder.edap-cluster.com/api/search?query='
            url_end = r'&order=asc'
            api_key = os.environ.get('apiKey')
            t.sleep(0.25) #Courtesy sleep time because we're respectful in this house
            response = requests.get(url_start+record_hash+url_end, headers = {'X-API-Key': api_key})
            json = response.json()
#Look through the retrieved json for the clowder id and file name.
            clowder_id = json['results'][0]['id']
            file_name = json['results'][0]['name']
            key.loc[i,'clowder_id'] = clowder_id
            key.loc[i,'file_name'] = file_name
            key.loc[i,'record_hash'] = record_hash
            key.loc[i,'found'] = 'found'
#Check to see how many new matches were found (if any). I do not expect many, so lets see what they are
print(f'{len(new_matches)} new matches have been found based on long ref fields')
print(new_matches)

In [ ]:
#After all of the checks and matches, lets create the final key for efsa that contains all info from efsa raw
#as well as the clowder id and file name fields.
key = key.fillna('')

#Columns from the old key to keep
stuff = {'source_hash': key['source_hash'],
         'name': key['name'],
         'casrn': key['casrn'],
         'record_url':key['record_url'], 
         'long_ref':key['long_ref'],
         'year':key['year'],
         'found':key['found'],
         'clowder_id':key['clowder_id'],
         'file_name':key['file_name']
        }

#Make the final key with the data from the old key
final_key = pd.DataFrame(data = stuff)
#Save the final key as an excel file so I can double check it.
out_path = r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\efsa_final_key_mmille16_09212022.xlsx"
with pd.ExcelWriter(out_path) as writer:
    final_key.to_excel(writer, index = False)

In [ ]:
#Now we return to the raw file and complete the task

for i in range(len(raw)):
    #Lets match the url in raw to the url in the key
    raw_url = raw.loc[i,'record_url']
    row = final_key[final_key['record_url'].str.match(raw_url)]
    #Then fill in those clowder id and pdf name fields
    raw.loc[i,'clowder_id'] = row.loc[row.index[0],'clowder_id']
    raw.loc[i,'pdf_name'] = row.loc[row.index[0],'file_name']

#Change all the nan entries to blank strings if there are any.
raw = raw.fillna('')

#Fill in found/not found based on clowder id entries
#Redoing this because the field was populated differently for different errors thrown during the key creation process
for i in range(len(raw)):
    if raw.loc[i,'clowder_id'] == '':
        raw.loc[i,'found'] = 'not found'
    else:
        raw.loc[i,'found'] = 'found'

#Taking stock of what records were and wer not matched
arr = raw['found']
num1 = list(arr).count('found')
num2 = list(arr).count('not found')
total = len(raw)

#Turn found/not found into percents and print out an update on the status of efsa raw.
percent_found = (num1/total)*100
percent_missing = (num2/total)*100
print(f'{percent_found}% of records in raw have been found and matched to documents in clowder')
print(f'{percent_missing}% of records in raw are missing and need to be uploaded to clowder')

#Save the updated raw file with matches to an excel file
out_path = r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\source_efsa_matched_mmille16_09212022.xlsx"
with pd.ExcelWriter(out_path) as writer:
    raw.to_excel(writer, index = False)

In [ ]:
#Use the final key to output an excel spreadsheet with only the uniqe records that were not found

#Initialize the dataframe
d = {}
unique_missing = pd.DataFrame(data = d)
j = 0
final_key = final_key.fillna('')
#Populate it with any key entries with empty clowder id fields
for i in range(len(final_key)):
    if final_key.loc[i,'clowder_id'] == '':
        for column in final_key.columns:
            unique_missing.loc[j,column] = final_key.loc[i,column]
        j += 1

#Save the missing key entries dataframe as an excel sheet
out_path = r"L:\PRIV\ToxValDB\Document Mapping\source_efsa\source_efsa_missing_mmille16_09212022.xlsx"
with pd.ExcelWriter(out_path) as writer:
    unique_missing.to_excel(writer, index = False)